In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = r"D:\guvi\data\emotion\train"
test_dir =  r"D:\guvi\data\emotion\test"
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.3,
    zoom_range=0.3, 
    horizontal_flip=True,
    fill_mode='nearest')

In [3]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)
val_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
len(train_generator),len(val_generator)

(898, 225)

In [11]:
image_batch, label_batch = next(train_generator)
print(image_batch.shape, label_batch.shape)

(32, 224, 224, 3) (32, 7)


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization,Activation
from tensorflow.keras.regularizers import l2
import warnings
warnings.filterwarnings('ignore')

In [18]:
model = Sequential()
model.add(Conv2D(16, (3,3),padding='same',kernel_regularizer=l2(0.001), input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3),padding='same',kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(128,kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
          

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 224, 224, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 224, 224, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 224, 224, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 112, 112, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 112, 112, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │      12,845,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,851,879 (49.03 MB)

 Trainable params: 12,851,527 (49.02 MB)

 Non-trainable params: 352 (1.38 KB)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    restore_best_weights=True)

In [20]:
print(model.input_shape)  
print(model.output_shape) 

(None, 224, 224, 3)
(None, 7)


In [21]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator,callbacks=[early_stopping])

Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1195s 1s/step - accuracy: 0.2285 - loss: 2.1625 - val_accuracy: 0.2409 - val_loss: 2.0973
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1185s 1s/step - accuracy: 0.2517 - loss: 2.0658 - val_accuracy: 0.2558 - val_loss: 2.0707
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1204s 1s/step - accuracy: 0.2522 - loss: 2.0588 - val_accuracy: 0.2509 - val_loss: 2.0450
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1239s 1s/step - accuracy: 0.2588 - loss: 2.0393 - val_accuracy: 0.2654 - val_loss: 2.0277
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1205s 1s/step - accuracy: 0.2615 - loss: 2.0248 - val_accuracy: 0.2232 - val_loss: 2.1117
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1121s 1s/step - accuracy: 0.2604 - loss: 2.0188 - val_accuracy: 0.2558 - val_loss: 2.0278
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1097s 1s/step - accuracy: 0.2610 - loss: 2.0068 - val_accuracy: 0.2510 - val_loss: 2.0134
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 1099s 1s/step - accuracy: 0.2700 - loss: 1.9996 - 

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()